## Online ingestion observability

This API provides the possibility to wait for online for online ingestion to finish.

## Scope

* Create and insert into online enabled feature group
* Get online ingestion
* Use online ingestion object

In [ ]:
import hopsworks

## Connect to the cluster

In [ ]:
# Connect to your cluster, to be used running inside Jupyter or jobs inside the cluster.
project = hopsworks.login()

In [ ]:
# Uncomment when connecting to the cluster from an external environment.
# project = hopsworks.login(project='my_project', host='my_instance', port=443, api_key_value='apikey')

## Create and insert into online enabled feature group.

In [ ]:
# Generate simple dataframe
import pandas as pd
size = 10
fg_data = {'id': range(0, size), 'text': "test"}
fg_df = pd.DataFrame.from_dict(fg_data)

In [ ]:
# Get project feature store
fs = project.get_feature_store()

In [ ]:
# Get/create feature group
fg = fs.get_or_create_feature_group(name="fg", version=1, primary_key=["id"], online_enabled=True)

In [ ]:
# Create a feature group and insert it into it, waiting for online and offline ingestion to finish
fg.insert(fg_df, write_options={"wait_for_online_ingestion": True, "wait_for_job": True})

## Get online ingestion

In [ ]:
# Get the latest ingestion associated with the feature group
online_ingestion_instance = fg.get_latest_online_ingestion()

In [ ]:
# Get specific ingestion by its id
#online_ingestion_instance = fg.get_online_ingestion(1)

## Use online ingestion object

In [ ]:
# Wait for online ingestion to finish (same as setting option `wait_for_online_ingestion` to true when inserting)
online_ingestion_instance.wait_for_completion(options={"timeout": 0, "period": 1})

In [ ]:
# Print results of ingestion (if there is only a result with `UPSERTED` status and rows match the expected data size then ingestion did not encounter any issues)
# ex. [{'onlineIngestionId': 1, 'status': 'UPSERTED', 'rows': 10}]
print([result.to_dict() for result in online_ingestion_instance.results])

In [ ]:
# Get logs from the online ingestion service (useful to check what issues were encountered while ingesting data)
online_ingestion_instance.print_logs(priority="error", size=5)